# Lecture 12 – Simple Linear Regression

**by Suraj Rampure**

Notebook credits:
- Ani Adhikari
- Data 8's textbook, chapter 15

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

### Correlation

In [2]:
def standard_units(x):
    return (x - np.mean(x)) / np.std(x)

def correlation(x, y):
    return np.mean(standard_units(x) * standard_units(y))

Let's read in our data.

In [26]:
heights = pd.read_csv('data/family_heights.csv').iloc[:, 1:]

In [22]:
heights

,father,mother,child,children,order,sex
0,78.5,67.0,73.2,4,1,male
1,78.5,67.0,69.2,4,2,female
2,78.5,67.0,69.0,4,3,female
3,78.5,67.0,69.0,4,4,female
4,75.5,66.5,73.5,4,1,male
...,...,...,...,...,...,...
929,62.0,66.0,64.0,3,1,male
930,62.0,66.0,62.0,3,2,female
931,62.0,66.0,61.0,3,3,female
932,62.5,63.0,66.5,2,1,male


In [27]:
heights["parent ave height"] = (heights["father"]+heights["mother"])/2

heights.head()

,father,mother,child,children,order,sex,parent ave height
0,78.5,67.0,73.2,4,1,male,72.75
1,78.5,67.0,69.2,4,2,female,72.75
2,78.5,67.0,69.0,4,3,female,72.75
3,78.5,67.0,69.0,4,4,female,72.75
4,75.5,66.5,73.5,4,1,male,71.00


In [29]:
fig = px.scatter(heights, x= "parent ave height", y = 'child')
fig.show()

Using our `correlation` function:

In [ ]:
correlation(df['parent'], df['child'])

Using an in-built `correlation` function:

In [ ]:
np.corrcoef(df['parent'], df['child'])

In [ ]:
df.corr()

All the same result.

What we now want to do is compute the **average $y$ for a given $x$**. A practical way to do this is to "bin" our x axis into 1-unit wide buckets, and then compute the average $y$ value for everything in that bucket. (We could choose bins of any width, though.)

In [ ]:
def predict_mean_y(x):
    return df.loc[np.abs(df['parent'] - x) <= 0.5, 'child'].mean()

In [ ]:
df['child_predicted'] = df['parent'].apply(predict_mean_y)

Now, let's look at our predictions:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df['parent'], y = df['child'], mode = 'markers', name = 'actual'))
fig.add_trace(go.Scatter(x = df['parent'], y = df['child_predicted'], mode = 'markers', name = 'predicted means', line=dict(color='gold')))
fig.update_layout(xaxis_title = 'MidParent Height', yaxis_title = 'Child Height')

Save for the tails where there are fewer values to draw from, it seems like our red predictions roughly follow a straight line piercing through the "middle" of our point cloud. That's our motivation for using a line to model this bivariate data.

Note: The cool thing about plotly is that you can hover over the points and it will tell you whether it is a prediction or actual value.

Now, it's time to implement the optimal coefficients.

In [ ]:
def slope(x, y):
    return correlation(x, y) * np.std(y) / np.std(x)

def intercept(x, y):
    return np.mean(y) - slope(x, y)*np.mean(x)

In [ ]:
ahat = intercept(df['parent'], df['child'])
bhat = slope(df['parent'], df['child'])

print("predicted y = {} + {} * average parent's height".format(np.round(ahat, 2), np.round(bhat, 2)))

Let's see what our linear model looks like.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df['parent'], y = df['child'], mode = 'markers', name = 'actual'))
fig.add_trace(go.Scatter(x = df['parent'], y = df['child_predicted'], mode = 'markers', name = 'predicted means', line=dict(color='gold')))
fig.add_trace(go.Scatter(x = df['parent'], y = ahat + bhat*df['parent'], name = 'linear model', line=dict(color='red')))


fig.update_layout(xaxis_title = 'MidParent Height', yaxis_title = 'Child Height')

## Visualizing Loss Surface

Let's look at what the loss surface for the above model looks like. Don't worry too much about what this code is doing.

In [ ]:
def mse(y, yhat):
    return np.mean((y - yhat)**2)

In [ ]:
# This function takes in our choice of [a, b] as a list, and returns
# the MSE for the corresponding linear model
def mse_for_height_model(t):
    a, b = t
    return mse(df['child'], a + b*df['parent'])

In [ ]:
num_points = 200 # increase for better resolution, but it will run more slowly. 

# if (num_points <= 100):

uvalues = np.linspace(20, 32, num_points)
vvalues = np.linspace(-1, 3, num_points)
(u,v) = np.meshgrid(uvalues, vvalues)
thetas = np.vstack((u.flatten(),v.flatten()))

MSE = np.array([mse_for_height_model(t) for t in thetas.T])

loss_surface = go.Surface(x=u, y=v, z=np.reshape(MSE, u.shape))

opt_point = go.Scatter3d(x = [ahat], y = [bhat], z = [mse_for_height_model((ahat, bhat))],
            mode = 'markers', name = 'optimal parameters',
            marker=dict(size=10, color='gold'))

fig = go.Figure(data=[loss_surface])
fig.add_trace(opt_point)

fig.update_layout(scene = dict(
    xaxis_title = "theta0",
    yaxis_title = "theta1",
    zaxis_title = "MSE"))


fig.show()
# else:
#     print("Picking num points > 100 can be really slow. If you really want to try, edit the code above so that this if statement doesn't trigger.")

As you can see, our choice of $\hat{a}, \hat{b}$ truly do minimize mean squared error. They exist at the minimum value of the loss surface.

## Multiple Linear Regression

Let's load in a new dataset. This is aggregate per-player data from the 2018-19 NBA season.

In [ ]:
nba = pd.read_csv('nba18-19.csv')

In [ ]:
nba

Let's suppose our goal is to predict the number of points someone averaged (`PTS`; this is our dependent variable). The independent variables we'll use are
- `AST`, the average number of assists per game, and
- `3PA`, the number of 3 point field goals attempted per game

In [ ]:
nba[['AST', '3PA', 'PTS']]

First, let's explore and fit a model using just `AST`.

In [ ]:
plt.scatter(nba['AST'], nba['PTS'])
plt.xlabel('average assists per game')
plt.ylabel('average points per game');

In [ ]:
r_nba = correlation(nba['AST'], nba['PTS'])
r_nba

The correlation between `AST` and `PTS` is relativelty strong. However, the scatter plot above tells us this isn't exactly the optimal setting in which to perform linear regression. For the purposes of illustration, we'll continue with it anyways.

In [ ]:
ahat_nba = intercept(nba['AST'], nba['PTS'])
bhat_nba = slope(nba['AST'], nba['PTS'])

print("predicted PTS = {} + {} * AST".format(np.round(ahat_nba, 2), np.round(bhat_nba, 2)))

Let's take a look at our prediction:

In [ ]:
pred_ast_only = ahat_nba + bhat_nba*nba['AST']

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = nba['AST'], y = nba['PTS'], mode = 'markers', name = 'actual'))
fig.add_trace(go.Scatter(x = nba['AST'], y = pred_ast_only, name = 'predictions', line=dict(color='red')))


fig.update_layout(xaxis_title = 'AST', yaxis_title = 'PTS')

Our model does _okay_. Let's compute the RMSE (that is, the square root of the mean squared error; we take the square root so that the RMSE is in the same units as our $y$ values). We will use this as a baseline for when we add more indepedent variables.

In [ ]:
def rmse(y, yhat):
    return mse(y, yhat)**0.5

In [ ]:
rmse_ast_only = rmse(nba['PTS'], pred_ast_only)

In [ ]:
rmse_ast_only

There's still a ton of variation in our model. Let's see if we can do better, by incorporating `3PA` as well (that is, the average number of 3 point shot attempts they made per game).

Specifically, we're looking to create the model

$$\text{predicted PTS} = \theta_0 + \theta_1 \cdot \text{AST} + \theta_2 \cdot \text{3PA}$$

In orrder to do this, we're going to import a new library, called `sklearn`. Don't worry too much about what it's doing for now – we will dedicate an entire section of lecture to it in 2 lectures from now.

In [ ]:
import sklearn.linear_model as lm

In [ ]:
model = lm.LinearRegression(fit_intercept = True)

model.fit(nba[['AST', '3PA']], nba['PTS']);

In [ ]:
model.coef_

In [ ]:
model.intercept_

The above outputs tell us that the parameters that minimize MSE for this model are
- $\theta_0 = 2.1563$
- $\theta_1 = 1.6407$
- $\theta_2 = 1.2576$

Meaning our predictions should be of the form

$$\text{predicted PTS} = 2.1563 + 1.6407 \cdot \text{AST} + 1.2576 \cdot \text{3PA}$$

Let's visualize what our model and predictions look like.

In [ ]:
pred_ast_3pa = model.predict(nba[['AST', '3PA']])

In [ ]:
fig = go.Figure()

nba_data_raw = go.Scatter3d(x = nba['AST'], y = nba['3PA'], z = nba['PTS'],
            mode = 'markers', marker=dict(color='cornflowerblue', size = 5), name = 'actual')

num_points = 100
xvalues = np.linspace(0, 11, num_points)
yvalues = np.linspace(0, 10, num_points)
(u,v) = np.meshgrid(xvalues, yvalues)
ast3pa = np.vstack((u.flatten(),v.flatten()))

pred_pts = np.array([model.predict(coord.reshape(1, -1)) for coord in ast3pa.T])

loss_surface = go.Surface(x=u, y=v, z=np.reshape(pred_pts, u.shape), name = 'predicted')

# prediction_surface = go.Scatter3d(x = nba['AST'], y = nba['3PA'], z = pred_ast_3pa, line = dict(color='gold'),
#                                  mode = 'markers')

fig.add_trace(loss_surface)
fig.add_trace(nba_data_raw)

fig.update_layout(scene = dict(
    xaxis_title = "AST",
    yaxis_title = "3PA",
    zaxis_title = "PTS"))

fig.show()

Instead of our model being a line, it is now a plane in 3D (the colorful surface above). The blue points above are the true `PTS` values.

It's sometimes hard to interpret things in 3D; we can also visualize in 2D.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = nba['AST'], y = nba['PTS'], mode = 'markers', name = 'actual'))
fig.add_trace(go.Scatter(x = nba['AST'], y = pred_ast_only, name = 'lm AST only', line=dict(color='red')))
fig.add_trace(go.Scatter(x = nba['AST'], y = pred_ast_3pa, mode = 'markers', name = 'lm AST 3PA', line=dict(color='gold')))


fig.update_layout(xaxis_title = 'average assists per game', yaxis_title = 'points per game')

The yellow dots are the result of our updated linear model. It doesn't look linear here, because it is not solely a function of assists per game. (It was linear in the 3D figure above.) The yellow points here all lie on the colorful plane above.

We can also scatter our predicted values vs. our actual values.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = nba['PTS'], y = pred_ast_only, mode = 'markers', name = 'lm AST only', line=dict(color='red')))
fig.add_trace(go.Scatter(x = nba['PTS'], y = pred_ast_3pa, mode = 'markers', name = 'lm AST 3PA', line=dict(color='gold')))


fig.update_layout(xaxis_title = 'actual points per game', yaxis_title = 'predicted points per game')

Let's also look at our RMSE.

In [ ]:
pred_ast_3pa = model.predict(nba[['AST', '3PA']])

rmse_ast_3pa = rmse(nba['PTS'], pred_ast_3pa)
rmse_ast_3pa

It's noticably lower than before!

In [ ]:
rmse_ast_only

## Multiple $R^2$

In [ ]:
r2_ast_only = np.var(pred_ast_only) / np.var(nba['PTS'])
r2_ast_3pa = np.var(pred_ast_3pa) / np.var(nba['PTS'])

In [ ]:
r2_ast_only

In [ ]:
correlation(pred_ast_only, nba['PTS'])**2

In [ ]:
r2_ast_3pa

In [ ]:
correlation(pred_ast_3pa, nba['PTS'])**2

This means that our model that only uses `AST` can explain 45% of the variation of the true observations (`PTS` values), while our model that uses `AST` and `3PA` can explain 60% of the variation. 